In [3]:
from typing import Dict
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from plot import plot_hbar_groups

In [4]:
COLORS = {
    "chainguard": "blue",
    "rapidfort": "firebrick",
    "alpine": "pink",
    "original": "gray"
}

def format_df(df: pd.DataFrame, column: str) -> Dict:
    registries = df["registry_type"].unique()
    images = df["image_type"].unique()
    members = {r: [] for r in registries}

    for reg in registries:
        for img in images:
            tmask = df["registry_type"] == reg
            imask = df["image_type"] == img
            filtered = df[tmask & imask][column]
            if filtered.shape[0] == 0:
                val = 0
            else:
                val = filtered.iloc[0]
            members[reg].append(val)
    
    return members, images

def plot_df(df: pd.DataFrame, column: str):
    _, ax = plt.subplots(figsize=(10,20))
    members, groups = format_df(df, column)
    plot_hbar_groups(ax, members, groups, COLORS)

In [6]:
# Image size by registry
sz_df = pd.read_csv(os.path.join("data", "out", "metadata.csv"))
sz_df["image_size"] = sz_df["image_size"] / 1000000

comps_df = pd.read_csv(os.path.join("data", "out", "components.csv"))
comps_df = comps_df.drop(["type"], axis=1) \
                    .groupby(["registry_type", "image_type"]) \
                    .count() \
                    .reset_index() \
                    .rename(columns={"component_name": "n_components"})

vulns_df = pd.read_csv(os.path.join("data", "out", "vulns.csv"))
vulns_df = vulns_df.drop(["type"], axis=1) \
                   .groupby(["registry_type", "image_type"]) \
                   .count() \
                   .reset_index() \
                   .rename(columns={"severity": "n_vulns"})

df = vulns_df.merge(sz_df, how="left",
                    on=["registry_type", "image_type"])
df = df.merge(comps_df, how="left",
              on=["registry_type", "image_type"])
df["vpm"] = df["n_vulns"] / df["image_size"]
df["vpc"] = df["n_vulns"] / df["n_components"]

df.groupby("registry_type").describe()

FileNotFoundError: [Errno 2] No such file or directory: 'data/out/metadata.csv'